In [ ]:
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pandas as pd
import pickle
from dataclasses import dataclass

In [ ]:
app = FastAPI()


In [ ]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
@dataclass
class VARIABLES:
    model_filename = './model.pkl'


In [ ]:
class ModelInput(BaseModel):
    # write model input columns
    Annual_Income: Optional[float64] = None
    Monthly_Inhand_Salary: Optional[float64] = None
    Num_Bank_Accounts: Optional[float64] = None
    Num_Credit_Card: Optional[float64] = None
    Interest_Rate: Optional[float64] = None
    Delay_from_due_date: Optional[float64] = None
    Num_of_Delayed_Payment: Optional[float64] = None
    Changed_Credit_Limit: Optional[float64] = None
    Outstanding_Debt: Optional[float64] = None
    Total_EMI_per_month: Optional[float64] = None
    Amount_invested_monthly: Optional[float64] = None
    Credit_Mix: Optional[float64] = None
    Payment_of_Min_Amount_No: Optional[float64] = None
    Payment_of_Min_Amount_Yes: Optional[float64] = None



In [ ]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    data = data.drop(columns = ['Unnamed: 0','Type_of_Loan', 'Payment_Behaviour','ID', 'Age','SSN','Customer_ID', 'Occupation','Name', 'Month'], axis = 1)
    data['Credit_Score'] = data['Credit_Score'].replace({"Poor": 0, "Good": 1, "Standard":2})
    data_num=data.select_dtypes(exclude = 'object')
    data_obj=data.select_dtypes(include = 'object')
    data_obj = pd.get_dummies(data_obj, columns=['Payment_of_Min_Amount'], drop_first = True)
    le = LabelEncoder()
    data_obj['Credit_Mix'] = le.fit_transform(data_obj.Credit_Mix)
    scaler = MinMaxScaler()
    data_num_standardized = pd.DataFrame(scaler.fit_transform(data_num), 
                                        columns=data_num.columns, index=data_num.index)
    data = pd.concat([data_num_standardized, data_obj], axis = 1)


    pass

In [ ]:
@app.get("/predict")
def root(x_model: ModelInput):
    data = pd.DataFrame(x_model)
    data = preprocess(data)
    result = model.predict(data)
    return {
        "the result of model": result
    }